In [ ]:
# 学習済みのベクトルデータの準備
import gdown
if not os.path.exists('/content/data'):
    os.mkdir('/content/data')

url = 'https://drive.google.com/uc?id=1BwRe4FnPi26qzpUZ1r98x9PVkWLaA1d6'
gdown.download(url, './data/stem_vec.tar.gz', quiet=False)
!tar zxvf /content/data/stem_vec.tar.gz


In [ ]:
# 類義語を検索してみる
import gensim
import warnings
from gensim.models import Word2Vec

if not os.path.exists('/content/stem_vec/word2vec.gensim.egotw.model'):
    raise ValueError("ERROR: 学習済みword2vecモデルが存在しません。")

# 学習済みWord2vecを読込
wv = Word2Vec.load("/content/stem_vec/word2vec.gensim.egotw.model")

# 検索
input_word = '学生'
if not input_word in wv:
    print("Input word [%s] is not included in the model" % input_word)
else:
    results = wv.most_similar(positive=[input_word])
    for result in results:
        print(result)

In [ ]:
# word2vecの学習を追加で行う

# 形態素解析器MeCabのラッパーライブラリfugashiをインストール
!pip install fugashi[unidic-lite]
from fugashi import Tagger

txt = '私は徳島大学の学生です。'

tagger = Tagger('-Owakati')

# 形態素解析する関数
def wakatigaki(txt):
    words = []
    wakati = tagger.parse(txt)
    for w in wakati.split(' '):
        words.append(w)
    return words

for w in wakatigaki(txt):
    print(w)

In [ ]:
# コーパスの準備
# テキストデータ(livedoor news corpus)をMeCab(fugashi)で分かち書き処理
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar zxvf ldcc-20140209.tar.gz
# テキストを1つのファイルにまとめる(すべてのファイルで3行目以降が本文)
import glob
wf = open('./all.txt', 'w')
for dir in glob.glob('/content/text/*'):
    for path in glob.glob(f'{dir}/*.txt'):
        l = 0
        for line in open(path):
            line = line.rstrip()
            if line == '':
                continue

            l += 1
            if l >= 3:
                wf.write(' '.join(wakatigaki(line)) + '\n')
wf.close


In [ ]:
# livedoor news corpusの分かち書きテキストを
# 追加で学習させる
from gensim.models import word2vec
corpus = '/content/all.txt'
newmodel = '/content/stem_vec/w2v.new.livedoor.model'
sentences = word2vec.Text8Corpus(corpus)

model = word2vec.Word2Vec.load("/content/stem_vec/word2vec.gensim.egotw.model")
model.build_vocab(sentences, update=True)
model.train(sentences, total_examples=model.corpus_count, epochs=10)
model.save(newmodel)

In [ ]:
# 青空文庫コーパスを追加学習する
#

url = 'https://drive.google.com/uc?id=1C0vS_Z7yojtblbg2YaRULv6Zh2Q1FxGz'
gdown.download(url, './data/author_novel.zip', quiet=False)
!unzip /content/data/author_novel.zip

wf = open('./all_aozora.txt', 'w')
for path in glob.glob('./author_novel/*'):
    for line in open(path):
        line = line.rstrip()
        if line == '':
            continue
        wf.write(' '.join(wakatigaki(line)) + '\n')
wf.close

In [ ]:
# 青空文庫コーパスの分かち書きテキストを
# 追加で学習させる
from gensim.models import word2vec
corpus = '/content/all_aozora.txt'
newmodel_aozora = '/content/stem_vec/w2v.new.aozora.model'
sentences = word2vec.Text8Corpus(corpus)

model = word2vec.Word2Vec.load("/content/stem_vec/word2vec.gensim.egotw.model")
model.build_vocab(sentences, update=True)
model.train(sentences, total_examples=model.corpus_count, epochs=10)
model.save(newmodel_aozora)

In [ ]:
# 追加学習したWord2vecを読込
wvnew = Word2Vec.load(newmodel)

# 検索
input_word = '学生'
if not input_word in wvnew:
    print("Input word [%s] is not included in the model" % input_word)
else:
    results = wvnew.most_similar(positive=[input_word])
    for result in results:
        print(result)

wvnew_aozora = Word2Vec.load(newmodel_aozora)
# 検索
input_word = '学生'
if not input_word in wvnew_aozora:
    print("Input word [%s] is not included in the model" % input_word)
else:
    results = wvnew_aozora.most_similar(positive=[input_word])
    for result in results:
        print(result)